# 基于预训练模型的槽位填充和意图识别

本示例将展示如何使用ERNIE预训练模型完成任务行对话中的槽位填充和意图识别任务，这两个任务是一个pipline型任务对话系统的基石。

本示例使用的数据集为CrossWOC中文对话数据集。该数据集包含多个领域，包括景点，餐馆，酒店，交通等。

原始数据和数据处理脚本请参见 [CrossWOZ](https://github.com/thu-coai/CrossWOZ)。

## 环境要求

* PaddlePaddle

   本项目依赖于 PaddlePaddle 2.0 及以上版本，请参考 [安装指南](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/pip/windows-pip.html) 进行安装

* PaddleNLP 

   ```shell
   pip install --upgrade paddlenlp -i https://pypi.org/simple
   ```

* Python

    Python的版本要求 3.6+


AI Studio平台后续会默认安装PaddleNLP，在此之前可使用如下命令安装

In [1]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

     |████████████████████████████████| 378kB 253kB/s eta 0:00:01
     |████████████████████████████████| 112kB 118kB/s eta 0:00:01
  Found existing installation: paddlenlp 2.0.0rc7
    Uninstalling paddlenlp-2.0.0rc7:
      Successfully uninstalled paddlenlp-2.0.0rc7


# 示例流程

与大多数NLP任务相同，本次示例的展示流程分为以下四步：

首先我们从数据准备开始。

![](https://ai-studio-static-online.cdn.bcebos.com/dd30e17318fb48fabb5701fd8a97be8176a1e372dd134cc0826e58cb5401933d)

# 数据准备

数据准备流程如下：

![](https://ai-studio-static-online.cdn.bcebos.com/89ba02da6f914297ae2fc438d1c9f773556f226652134fb684ac0186bfa5bb7d)

## 1. 使用`load_dataset()`自定义数据集

使用官方脚本预处理过的数据集已经上传至项目根目录，观察数据集格式后我们可以根据数据格式写出数据文件读取函数，传入`load_dataset()`。即可创建数据集。

In [2]:
from paddlenlp.datasets import load_dataset
import json

# 读取标签文件并创建label_map
def get_label_map(filename):
    with open(filename, "r", encoding="utf8") as f:
        label_list = json.load(f)
    
    id2label = dict([(idx, label) for idx, label in enumerate(label_list)])
    label2id = dict([(label, idx) for idx, label in enumerate(label_list)])
    return id2label, label2id

id2slot, slot2id = get_label_map('slot_labels.json')
id2intent, intent2id = get_label_map('intent_labels.json')

intent_weight = [1] * len(intent2id)

# 根据本地文件格式定义数据读取生成器
def read(filename):
    with open(filename, "r", encoding="utf8") as f:
        data = json.load(f)
    for entry in data:
        yield {
            'words': entry[0],
            'slots': entry[1],
            'intents': entry[2],
            'history': entry[4],
        }

# 将生成器传入load_dataset
train_ds = load_dataset(read, filename='train.json', lazy=False)
dev_ds = load_dataset(read, filename='test.json', lazy=False)

for idx in range(2):
    print(train_ds[idx]['words'])
    print(train_ds[idx]['slots'])
    print(train_ds[idx]['intents'])
    print(train_ds[idx]['history'])
    print()

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:26: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  def convert_to_list(value, n, name, dtype=np.int):


['你', '好', '，', '麻', '烦', '帮', '我', '推', '荐', '一', '个', '门', '票', '免', '费', '的', '景', '点', '。']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B+Inform+景点+门票', 'I+Inform+景点+门票', 'O', 'O', 'O', 'O']
['General+greet+none+none', 'Request+景点+名称+']
[]

['你', '好', '，', '您', '可', '以', '选', '择', '故', '宫', '，', '八', '达', '岭', '长', '城', '，', '颐', '和', '园', '或', '者', '红', '砖', '美', '术', '馆', '。']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B+Inform+景点+名称', 'I+Inform+景点+名称', 'I+Inform+景点+名称', 'I+Inform+景点+名称', 'I+Inform+景点+名称', 'O']
['General+greet+none+none']
['你好，麻烦帮我推荐一个门票免费的景点。']



关于更多自定义数据集相关内容，请参考[如何自定义数据集](https://paddlenlp.readthedocs.io/zh/latest/data_prepare/dataset_self_defined.html)

## 2. 加载 `paddlenlp.transformers.ErnieTokenizer`用于数据处理

文本数据在输入ERNIE预训练模型之前，需要经过数据处理转化为Feature。这一过程通常包括分词，token to id，add special token等步骤。  

**PaddleNLP对于各种预训练模型已经内置了相应的tokenizer**，指定想要使用的模型名字即可加载对应的tokenizer。

可以通过调用tokenizer中的方法简单的完成上述数据处理。

In [3]:
import paddlenlp

# 设置模型名称
MODEL_NAME = 'ernie-1.0'
tokenizer = paddlenlp.transformers.ErnieTokenizer.from_pretrained(MODEL_NAME)

[2021-05-31 14:58:44,883] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 4006.15it/s]


## 3. 调用`map()`方法批量处理数据

由于我们传入了`lazy=False`，所以我们使用`load_dataset()`自定义的数据集是`MapDataset`对象。

`MapDataset`是`paddle.io.Dataset`的功能增强版本。其内置的`map()`方法适合用来进行批量数据集处理。

`map()`方法传入的是一个用于数据处理的function。正好可以与tokenizer相配合。

以下是本示例中的用法：

In [4]:
from functools import partial
import numpy as np

def convert_example(example, tokenizer, use_history=False, no_entity_id=0, max_seq_len=512, mode='train'):
    
    # 调用tokenizer的数据处理方法把文本转为id
    tokenized_input = tokenizer(
        example['words'],
        is_split_into_words=True,
        max_seq_len=max_seq_len)
    
    # 对槽位标签做截断，保证和input_id等长
    slot_labels = example['slots']
    if len(tokenized_input['input_ids']) - 2 < len(slot_labels):
        slot_labels = slot_labels[:len(tokenized_input['input_ids']) - 2]
    
    # 根据label_map将槽位标签转为id
    tokenized_input['slot_labels'] = [no_entity_id] + [slot2id[label] for label in slot_labels] + [no_entity_id]
    
    # 由于任务中的意图识别是多标签分类，需要把意图标签转为类似one-hot的格式
    intent_labels = np.zeros(len(intent2id), dtype='int64')
    for l in example['intents']:
        intent_labels[intent2id[l]] = 1
        if mode == 'train':
            # 统计训练集中每个意图的正样本数
            intent_weight[intent2id[l]] += 1
    tokenized_input['intent_labels'] = intent_labels
    
    # 将历史对话用[SEP]拼起来并转成id
    if use_history:
        tokenized_history = tokenizer(
            tokenizer.cls_token+tokenizer.sep_token.join(example['history']),
            max_seq_len=max_seq_len)
        tokenized_input['history_ids'] = tokenized_history['input_ids']
    else:
        tokenized_input['history_ids'] = []
    return tokenized_input

use_history = False
max_seq_length = 512

train_trans_func = partial(
        convert_example,
        tokenizer=tokenizer,
        mode='train',
        use_history=use_history,
        max_seq_len=max_seq_length)

dev_trans_func = partial(
        convert_example,
        tokenizer=tokenizer,
        mode='dev',
        use_history=use_history,
        max_seq_len=max_seq_length)

train_ds.map(train_trans_func, lazy=False)   
dev_ds.map(dev_trans_func, lazy=False)

# 根据意图的正样本数和总样本数为不同样本赋予不同的权重
for intent, intent_id in intent2id.items():
    neg_pos = (len(train_ds) - intent_weight[intent_id]) / intent_weight[intent_id]
    intent_weight[intent_id] = np.log10(neg_pos)


In [5]:
for idx in range(2):
    print(train_ds[idx]['input_ids'])
    print(train_ds[idx]['token_type_ids'])
    print(train_ds[idx]['slot_labels'])
    print(train_ds[idx]['intent_labels'])
    print()

[1, 226, 170, 4, 1298, 1934, 836, 75, 426, 1645, 7, 27, 232, 1039, 783, 453, 5, 561, 180, 12043, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0]
[1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0]

[1, 226, 170, 4, 892, 48, 22, 352, 790, 470, 915, 4, 643, 302, 1560, 84, 257, 4, 3540, 14, 509, 172, 123, 536, 1520, 188, 133, 774, 12043, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 4, 4, 4, 4, 0, 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

从以上结果可以看出，数据集中的example已经被转换成了模型可以接收的feature，包括input_ids、token_type_ids、slot_labels和intent_labels。
其中：

* `input_ids`: 表示输入文本的token ID。
* `token_type_ids`: 表示token所属的句子（Transformer类预训练模型支持单句以及句对输入）。
* `slot_labels`: 槽位标签，其长度与输入文本相同。
* `intent_labels`: 意图标签，是一个长度等于总便签数的列表，标签对应位置为1，其余为0.

更多有关数据处理的内容，请参考[数据处理](https://paddlenlp.readthedocs.io/zh/latest/data_prepare/data_preprocess.html)。

## 4. Batchify和数据读入

使用`paddle.io.BatchSampler`和`paddlenlp.data`中提供的方法把数据组成batch。

然后使用`paddle.io.DataLoader`接口多线程异步加载数据。

`batchify_fn`详解：

![](https://ai-studio-static-online.cdn.bcebos.com/30e43d4659384375a2a2c1b890ca5a995c4324d7168e49cebf1d2a1e99161f7d)


In [7]:
import paddle
from paddlenlp.data import Stack, Dict, Pad

batch_size = 20

# 初始化BatchSampler
train_batch_sampler = paddle.io.BatchSampler(
    train_ds, batch_size=batch_size, shuffle=True)

dev_batch_sampler = paddle.io.BatchSampler(
    dev_ds, batch_size=batch_size, shuffle=False)

# 定义batchify_fn
batchify_fn = lambda samples, fn=Dict({
    "input_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id), 
    "token_type_ids": Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    "slot_labels": Pad(axis=0, pad_val=0, dtype="int64"),
    "intent_labels": Stack(dtype="float32"),
    "history_ids": Pad(axis=0, pad_val=tokenizer.pad_token_id)
}): fn(samples)

# 初始化DataLoader
train_data_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_sampler=train_batch_sampler,
    collate_fn=batchify_fn,
    return_list=True)

dev_data_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_sampler=dev_batch_sampler,
    collate_fn=batchify_fn,
    return_list=True)

更多PaddleNLP内置的batchify相关API，请参考[collate](https://paddlenlp.readthedocs.io/zh/latest/source/paddlenlp.data.collate.html)。

到这里数据集准备就全部完成了，下一步我们需要组网并设计loss function。

![](https://ai-studio-static-online.cdn.bcebos.com/fdcb44a00ede4ce08ae2652931556fb58cc903f686bf491792489353d2800e7d)

# 模型结构

## 使用PaddleNLP一键加载预训练模型

以下项目以ERNIE为例，介绍如何将预训练模型多任务学习同时完成意图识别和槽位填充任务。

本例中的意图识别和槽位填充本质上是一个句子分类任务和一个序列标注任务。将两者的loss结合即可实现多任务学习。

![](https://ai-studio-static-online.cdn.bcebos.com/d9ff881921d74602acb6eb27c8523cb50285f07a7beb4a3cbfa1edbd9b3f9c5c)




In [8]:
from src.models import JointErnie

model = JointErnie.from_pretrained(MODEL_NAME, 
                                   intent_dim=len(intent2id), 
                                   slot_dim=len(slot2id), 
                                   dropout=0.1, 
                                   use_history=use_history)

### 设计loss function

JointErnie模型会取出ErnieModel的sequence_output接入一个输出维度为槽位类别数的线性层得到slot_logits，并将pooled_output接入一个输出维度为意图类别数的线性层得到intent_logit.

所以本示例中的loss由slot_loss和intent_loss两部分组成，我们需要自己定义loss function。

槽位填充相当于在每个token的位置进行一次多分类任务，意图识别相当于对整句话做一个多标签分类任务。所以设计的loss function如下：

In [7]:
class NLULoss(paddle.nn.Layer):
    def __init__(self, pos_weight):
        super(NLULoss, self).__init__()

        self.intent_loss_fn = paddle.nn.BCEWithLogitsLoss(pos_weight=paddle.to_tensor(pos_weight))
        self.slot_loss_fct = paddle.nn.CrossEntropyLoss()

    def forward(self, logits, slot_labels, intent_labels):
        slot_logits, intent_logits = logits

        slot_loss = self.slot_loss_fct(slot_logits, slot_labels)
        intent_loss = self.intent_loss_fn(intent_logits, intent_labels)

        return slot_loss + intent_loss

NameError: name 'paddle' is not defined

选择网络结构后，我们需要设置Fine-Tune优化策略。

![](https://ai-studio-static-online.cdn.bcebos.com/7eca6595f338409498149cb586c077ba4933739810cf436080a2292be7e0a92d)


## 设置Fine-Tune优化策略
适用于ERNIE/BERT这类Transformer模型的学习率为warmup的动态学习率。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2bc624280a614a80b5449773192be460f195b13af89e4e5cbaf62bf6ac16de2c" width="40%" height="30%"/> <br />
</p>
<br><center>图3：动态学习率示意图</center></br>

In [10]:
# 训练过程中的最大学习率
learning_rate = 3e-5 

# 训练轮次
epochs = 10

# 学习率预热比例
warmup_proportion = 0.0

# 权重衰减系数，类似模型正则项策略，避免模型过拟合
weight_decay = 0.0

max_grad_norm = 1.0

num_training_steps = len(train_data_loader) * epochs

# 学习率衰减策略
lr_scheduler = paddlenlp.transformers.LinearDecayWithWarmup(learning_rate, num_training_steps,
                                    warmup_proportion)

decay_params = [
    p.name for n, p in model.named_parameters()
    if not any(nd in n for nd in ["bias", "norm"])
]

# 定义优化器
optimizer = paddle.optimizer.AdamW(
    learning_rate=lr_scheduler,
    parameters=model.parameters(),
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params,
    grad_clip=paddle.nn.ClipGradByGlobalNorm(max_grad_norm))

现在万事俱备，我们可以开始训练模型。

![](https://ai-studio-static-online.cdn.bcebos.com/6975542d488f4f75b385fe75d574a3aaa8e208f5e99f4acd8a8e8aea3b85c058)

## 模型训练与评估

模型训练的过程通常有以下步骤：

1. 从dataloader中取出一个batch data
2. 将batch data喂给model，做前向计算
3. 将前向计算结果传给损失函数，计算loss。
4. loss反向回传，更新梯度。重复以上步骤。

每训练一个epoch后，程序对调用`evaluation()`方法分别计算两个任务的F1 score。

In [6]:
from src.utils import evaluation

criterion = NLULoss(intent_weight)
global_step = 0

for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        global_step += 1
        input_ids, token_type_ids, slot_labels, intent_labels, history_ids = batch
        logits = model(input_ids, token_type_ids, history_ids=history_ids)
        loss = criterion(logits, slot_labels, intent_labels)

        if global_step % 1000 == 0 :
            print("global step %d, epoch: %d, batch: %d, loss: %.5f" % (global_step, epoch, step, loss))

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.clear_grad()
    
    print('\nEval begin...')
    evaluation(model, dev_data_loader)

NameError: name 'NLULoss' is not defined

# 引入对话历史

在该数据集的原始论文中，提到了一种改进模型的手段。即将对话历史的特征传入模型。可以增强模型的表现。

这里给大家举个例子：

- **当前句**："好，玩累了周边有酒店可以休息吗？"

- **意图**："Request+景点+周边酒店"

- **历史**：["你好，帮我找一个20到50元之间的景点，谢谢。","八达岭长城怎么样？门票35元。"]

在本示例中，可以通过修改数据处理函数的的`use_history`参数，并修改`batchify_fn`和对应训练代码来控制是否使用这一策略。

关于该策略的详细信息和原始baseline模型,请参考[CrossWOZ](https://github.com/thu-coai/CrossWOZ)。

# 更多任务

关于更多任务型对话的示例和数据集，可以参考PaddleNLP中的[DGU](https://github.com/PaddlePaddle/PaddleNLP/tree/develop/examples/dialogue/dgu)

以上内容实现基于PaddleNLP，开源不易，希望大家多多支持~ 
**记得给[PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)点个小小的Star⭐**

GitHub地址：[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)
![](https://ai-studio-static-online.cdn.bcebos.com/a0e8ca7743ea4fe9aa741682a63e767f8c48dc55981f4e44a40e0e00d3ab369e)

**更多使用方法可参考PaddleNLP教程**

- [使用seq2vec模块进行句子情感分类](https://aistudio.baidu.com/aistudio/projectdetail/1283423)
- [使用预训练模型ERNIE优化情感分析](https://aistudio.baidu.com/aistudio/projectdetail/1294333)
- [使用BiGRU-CRF模型完成快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1317771)
- [使用预训练模型ERNIE优化快递单信息抽取](https://aistudio.baidu.com/aistudio/projectdetail/1329361)
- [使用Seq2Seq模型完成自动对联](https://aistudio.baidu.com/aistudio/projectdetail/1321118)
- [使用预训练模型ERNIE-GEN自动写诗](https://aistudio.baidu.com/aistudio/projectdetail/1339888)
- [使用TCN网络完成新冠疫情病例数预测](https://aistudio.baidu.com/aistudio/projectdetail/1290873)
- [自定义数据集实现文本多分类任务](https://aistudio.baidu.com/aistudio/projectdetail/1468469)

# 加入交流群，一起学习吧

现在就加入PaddleNLP的QQ技术交流群，一起交流NLP技术吧！

<img src="https://ai-studio-static-online.cdn.bcebos.com/d953727af0c24a7c806ab529495f0904f22f809961be420b8c88cdf59b837394" width="200" height="250" >